In [1]:
import os
import zipfile
import urllib.request
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.io import arff
import torch
from torch.utils.data import TensorDataset, DataLoader

# Directory where datasets will be downloaded and extracted
DATA_DIR = 'datasets'

# Ensure the dataset directory exists
os.makedirs(DATA_DIR, exist_ok=True)

def download_dataset(dataset_name, url):
    """
    Downloads and extracts a zip file containing the dataset.
    """
    zip_path = os.path.join(DATA_DIR, f"{dataset_name}.zip")
    extract_path = os.path.join(DATA_DIR, dataset_name)

    # Download the dataset
    print(f"Downloading {dataset_name} from {url}...")
    urllib.request.urlretrieve(url, zip_path)

    # Extract the zip file
    print(f"Extracting {dataset_name}...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Remove the zip file after extraction
    os.remove(zip_path)
    print(f"Dataset {dataset_name} extracted to {extract_path}.")
    return extract_path

def load_arff_data(file_path):
    """
    Loads ARFF file and converts it to a pandas DataFrame.
    """
    print(f"Loading ARFF file: {file_path}")
    data, meta = arff.loadarff(file_path)
    df = pd.DataFrame(data)
    return df

def preprocess_data(train_df, test_df, batch_size=64):
    """
    Preprocesses the data:
    - Splits the features and labels.
    - Normalizes the features.
    - Converts them into PyTorch tensors.
    - Creates DataLoaders for training, validation, and testing.
    """
    # Separate features and labels
    train_features = train_df.drop(columns=['target'])
    test_features = test_df.drop(columns=['target'])

    train_labels = train_df['target'].apply(lambda x: int(x)).values
    test_labels = test_df['target'].apply(lambda x: int(x)).values

    # Normalize features
    scaler = StandardScaler()
    train_features_normalized = scaler.fit_transform(train_features)
    test_features_normalized = scaler.transform(test_features)

    # Reshape the features into 3D arrays (samples, time_steps, dimensions)
    X_train = train_features_normalized.reshape(-1, 96, 1)
    X_test = test_features_normalized.reshape(-1, 96, 1)

    # Split test data into validation and test sets
    X_valid, X_test, y_valid, y_test = train_test_split(X_test, test_labels, test_size=0.50, random_state=42)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(train_labels, dtype=torch.int64)

    X_valid = torch.tensor(X_valid, dtype=torch.float32)
    y_valid = torch.tensor(y_valid, dtype=torch.int64)

    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.int64)

    # Output dataset shapes
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}")
    print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    # Create DataLoaders
    train_dataset = TensorDataset(X_train, y_train)
    valid_dataset = TensorDataset(X_valid, y_valid)
    test_dataset = TensorDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

    return train_loader, valid_loader, test_loader, len(np.unique(train_labels))

# Example usage for downloading, extracting, and preprocessing the ElectricDevices dataset
if __name__ == "__main__":
    # URL for the dataset (replace with the actual dataset you want to download)
    dataset_name = 'ElectricDevices'  # Replace with your dataset name
    dataset_url = 'https://timeseriesclassification.com/aeon-toolkit/ElectricDevices.zip'

    # Download and extract the dataset
    extract_path = download_dataset(dataset_name, dataset_url)

    # Load ARFF data
    train_file = os.path.join(extract_path, f'{dataset_name}_TRAIN.arff')
    test_file = os.path.join(extract_path, f'{dataset_name}_TEST.arff')

    # Load data into Pandas DataFrames
    train_df = load_arff_data(train_file)
    test_df = load_arff_data(test_file)

    # Preprocess the data
    train_loader, valid_loader, test_loader, n_classes = preprocess_data(train_df, test_df)

    # Output the number of classes
    print(f"Number of classes: {n_classes}")


Extracting ElectricDevices...
Dataset ElectricDevices extracted to datasets/ElectricDevices.
Loading ARFF file: datasets/ElectricDevices/ElectricDevices_TRAIN.arff
Loading ARFF file: datasets/ElectricDevices/ElectricDevices_TEST.arff
X_train shape: torch.Size([8926, 96, 1]), y_train shape: torch.Size([8926])
X_valid shape: torch.Size([3855, 96, 1]), y_valid shape: torch.Size([3855])
X_test shape: torch.Size([3856, 96, 1]), y_test shape: torch.Size([3856])
Number of classes: 7
